[![img/pythonista.png](img/pythonista.png)](https://www.pythonista.mx)

# Ejemplo de uso con *JSON* y *Jinja*.

En este capítulo se creará una aplicación que realice una búsqueda de datos:

* Regresando un objeto serializado en formato *JSON*.
* Desplegando una página *HTML* mediante el uso de plantillas de *Jinja*.

## La función ```encuentra()```.

La función ```encuentra()``` busca patrones de texto en los campos de un objeto de tipo ```dict``` con las claves indicadas. En caso de encontrar al menos una coincidencia, regresará ```True```.

```
encuentra(<patron>, <obj>, <campos>)
```

Donde:

* ```<patron>``` es una cadena de caracteres.
* ```<obj>``` es un objeto de tipo ```dict```.
* ```<campos>``` es un objeto iterable que contiene las claves del objeto ```<obj>```en las que se realizará la bísquedas.

In [1]:
encuentra = lambda patron, obj, campos: bool(sum([patron.casefold() in obj[campo].casefold() for campo in campos]))

**Ejemplos:**

* La siguiente celda ejecutará la función ```encuentra()``` para buscar la cadena ```'nez'``` que se ingresa como primer argumento, en el objeto ingresado como segundo argumento, para las claves definidas por ```['apellido', 'nombre']``` ingresado como tercer argumento. El resultado será ```True```.

In [2]:
encuentra('nez', 
          {'nombre': 'Juan',
           'apellido': 'Godínez',
           'correo': 'falsonez@falso.com'},
          ['apellido', 'nombre'])

True

* La siguiente celda ejecutará la función ```encuentra()``` para buscar la cadena ```'Juan'``` que se ingresa como primer argumento, en el objeto ingresado como segundo argumento, para las claves definidas por ```['correo']``` ingresado como tercer argumento. El resultado será ```False```.

In [3]:
encuentra('Juan',
          {'nombre': 'Juan',
           'apellido': 'Godínez',
           'correo': 'falsonez@falso.com'},
          ['correo'])

False

* La siguiente celda ejecutará la función ```encuentra()``` para buscar la cadena ```'n'``` que se ingresa como primer argumento, en el objeto ingresado como segundo argumento, para las claves definidas por ```['correo', 'nombre', 'apellido']``` ingresado como tercer argumento. El resultado será ```True```.

In [4]:
encuentra('n', 
          {'nombre': 'Juan',
           'apellido': 'Godínez',
           'correo': 'falsonez@falso.com'},
          ['correo', 'nombre', 'apellido'])

True

## La función ```buscar_archivo()```.

La función ```buscar_archivo()``` realizará las siguientes operaciones:

* Leerá un documento de texto localizado en una ruta específica que contenga una representacion de una colección de objetos de tipoo ```dict```.
* Regresará una colección de todos los objetos que coincidan con un patrón de texto dado en los campos indicados.

```
buscar_archivo(<patron>, <ruta>, <campos>)
```

In [5]:
def buscar_archivo(patron, ruta, campos):
    with open(ruta, 'tr') as f:
        base = eval(f.read())
    return [item for item in base if encuentra(patron, item, campos)]

**Ejemplo:**

El archivo [```data/alumnos.py```](data/alumnos.py) contiene la representación de un objeto tipo ```list``` que a su vez contiene una sucesión de objetos tipo ```dict``` con los identificadores:

* ```'cuenta'```
* ```'nombre'```
* ```'primer_apellido'```
* ```'segundo_apellido'```
* ```'carrera'```
* ```'semestre'```
* ```'promedio'```
* ```'al_corriente'```.

``` python
[{'cuenta': 1231221, 'al_corriente': False, 'carrera': 'Arquitectura', 'nombre': 'Pedro', 'primer_apellido': 'Solis', 'promedio': 7.8, 'semestre': 3, 'segundo_apellido': 'Cabañas'},
{'cuenta': 1231222, 'al_corriente': False, 'carrera': 'Actuaría', 'nombre': 'Yolanda', 'primer_apellido': 'Jiménez', 'segundo_apellido': 'Lerdo', 'promedio': 6, 'semestre': 3},
{'cuenta': 1231223, 'al_corriente': True, 'carrera': 'sistemas', 'nombre': 'Juan', 'primer_apellido': 'Ramos', 'segundo_apellido': 'Breña', 'promedio': 8.6, 'semestre': 9},
{'cuenta': 1231224, 'al_corriente': True, 'carrera': 'Derecho', 'nombre': 'Mayra Jimena', 'primer_apellido': 'Cervantes', 'segundo_apellido': 'Lisama', 'promedio': 9.2, 'semestre': 12}]
```
* Se utilizará la función ```buscar_archivo()``` para encontrar los elementos que coincidan con la cadena de búsqueda en los campos ```'nombre'```, ```'primer_apellido'``` y ```'segundo_apellido'``` dentro del archivo ```data/alumnos.py```.

In [6]:
campos = ('nombre', 'primer_apellido', 'segundo_apellido')
ruta = 'data/alumnos.py'

In [7]:
buscar_archivo('ed', ruta, campos)

[{'cuenta': 1231221,
  'al_corriente': False,
  'carrera': 'Arquitectura',
  'nombre': 'Pedro',
  'primer_apellido': 'Solis',
  'promedio': 7.8,
  'semestre': 3,
  'segundo_apellido': 'Cabañas'}]

In [8]:
buscar_archivo('z', ruta, campos)

[{'cuenta': 1231222,
  'al_corriente': False,
  'carrera': 'Actuaría',
  'nombre': 'Yolanda',
  'primer_apellido': 'Jiménez',
  'segundo_apellido': 'Lerdo',
  'promedio': 6,
  'semestre': 3}]

In [9]:
buscar_archivo('Derecho', ruta, campos)

[]

In [10]:
buscar_archivo('Ramos', ruta, campos)

[{'cuenta': 1231223,
  'al_corriente': True,
  'carrera': 'Sistemas',
  'nombre': 'Juan',
  'primer_apellido': 'Ramos',
  'segundo_apellido': 'Breña',
  'promedio': 8.6,
  'semestre': 9}]

## Aplicación de búsqueda.

La siguiente aplicación levantará un servicio web que regresará una búsqueda con la función ```buscar_archivo()``` si la cadena de búsqueda se ingresa como parte de la ruta.

* Si la ruta es ```/api/<cadena>```regresará el resultado de la búsqueda de ```<cadena>```  en formato *JSON*.

* Si la ruta es ```/despliega/<cadena>``` regresará un documento *HTML* desplegando el resultado de la búsqueda.

### Definición de la plantilla.

La plantilla [```templates/busqueda.html```](templates/busqueda.html), que contiene el siguiente código que desplegará los datos de cada elemento encontrado y su contenido es el siguiente.

``` html
<h1> Alumnos Encontrados:</h1>
<ul>
    {% for alumno in alumnos %}
    <li>{% for campo in ['nombre', 'primer_apellido', 'segundo_apellido'] %}
{{alumno[campo]}}
        {% endfor %}</li> 
    {% endfor %}
</ul>
```

### El código principal.

In [11]:
import json
from flask import Flask, jsonify, render_template

app =Flask(__name__)


@app.route('/api/<termino>')
def busqueda(termino):
    '''Regresa el resultado de la búsqueda en formato JSON'''
    return jsonify(buscar_archivo(str(termino), ruta, campos))

@app.route('/despliega/<termino>')
def despliega(termino):
    '''Despliega el resultado de la búsqueda en un documento HTML'''
    return render_template('busqueda.html',
                           alumnos=buscar_archivo(str(termino),
                           ruta,
                           campos))

**Advertencia:** Una vez ejecutada la siguiente celda, es necesario interrumpir el *kernel* de *Jupyter* para poder ejecutar el resto de las celdas de la *notebook*.

In [ ]:
#Si no se define el parámetro host, flask sólo será visible desde localhost
# app.run(host='localhost')
app.run(host="0.0.0.0", port=5000)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://10.0.2.15:5000/ (Press CTRL+C to quit)
10.0.2.2 - - [10/Feb/2022 20:17:30] "GET /api/Ramos HTTP/1.1" 200 -
10.0.2.2 - - [10/Feb/2022 20:17:46] "GET /despliega/Ramos HTTP/1.1" 200 -
10.0.2.2 - - [10/Feb/2022 20:17:55] "GET /api/n HTTP/1.1" 200 -
10.0.2.2 - - [10/Feb/2022 20:18:03] "GET /despliega/n HTTP/1.1" 200 -
10.0.2.2 - - [10/Feb/2022 20:18:13] "GET /api/Juárez HTTP/1.1" 200 -
10.0.2.2 - - [10/Feb/2022 20:18:18] "GET /despliega/Juárez HTTP/1.1" 200 -


http://localhost:5000/api/Ramos

http://localhost:5000/despliega/Ramos

http://localhost:5000/api/n

http://localhost:5000/despliega/n

http://localhost:5000/api/Juárez

http://localhost:5000/despliega/Juárez


<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2022.</p>